<a href="https://colab.research.google.com/github/j-d-0630/Pytorch/blob/main/quick_start.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch